## Tensorflow

---

### 数据流图

* 数据流图是一种可以用来定义计算结构的特殊的有向图

* 数据流图的本质是一组连接在一起的函数(可以定义计算公式，和运算架构)，每一个函数可以将输出传递给多个(0, 1, n)个级联的函数

* 组成成分

  * 节点（Op） : 代表对数据的运算和某种操作
    * `input` 节点 : 传入初始数据
    * `output` 节点 : 传出结果数据
    * 每一个Op可以接收张量作为参数，将值传递给下一个直接连接的节点(Op)
  * 边     : 对节点传入的的数据(有向边)

* 注意点

  1. 运算顺序 : 

     实际的运算顺序我们无从知晓，但是我们需要知道只要节点之间的运算是没有相关联的话，我们可以不考虑运算顺序的问题

  2. 任何节点都可以将数据传递给数据流图中的任何后继节点，无论两者之间发生了多少计算

  3. 定义好我们的数据流图之后,我们可以抽象的理解我们的的数据流图是一个有输入输出的离散构件

  4. 节点依赖关系

     * 我们利用节点的依赖关系可以有效的对必要的以来节点执行计算，没有必要执行对整个图的计算，这就是依赖关系的用处

       背后使用**栈**的方式来管理

       * 将所有的以来节点从最终输出节点开始一次入栈可以保证只计算依赖节点，节约计算时间

     * 依赖关系具有传递性

       * 直接依赖节点
       * 间接依赖节点

     * 循环依赖

       前一个节点依赖于后一个节点的输出的情况，会导致死锁问题(循环依赖的出现)

       不支持的原因

       1. 程序不能有效的终止
       2. 信息追踪不可能
       3. 结果上溢或者下溢或者0

       数据流图的展开可以实现有限次的循环依赖(模拟有用的循环依赖)

### Tensorflow的工作模式

* 模式

  1. 定义数据流图
  2. 运行数据流图

* 实例

  定义计算公式的数据流图

  $$ y = 5 * 3 + (5 + 3)$$

* 节点  
    * `tf.constant` : 定义常量节点
    * `tf.multiply` : 定义乘法节点
    * `tf.subtrace` : 定义减法节点
    * `tf.negative` : 定义相反数节点
    * `tf.div`      : 定义整数除法节点
    * `tf.truediv`  : 定义浮点数除法节点
    * `tf.mod`      : 定义取模节点
    * `tf.square`   : 定义平方计算节点
    * `tf.matmul`   : 定义矩阵乘法计算节点
* Session  
    * `tf.Session.graph` : 引用了数据流图的描述
    * `tf.Session.run`          : 对不同的节点执行运算,**返回numpy数组**
    * `tf.Session.close` : 关闭会话，释放资源
* writer  
    * `tf.summary.FileWriter(path, graph)` : 将数据流图的描述写入
    * `tensorboard --logdir='my_graph` : 可视化窗口查看数据流图
    * `writer.close` : 关闭释放资源

In [4]:
# 数据流图的定义,运行不会有任何输出
a = tf.constant(5, name = 'input_a')
b = tf.constant(3, name = 'input_b')
c = tf.multiply(a, b, name = 'mul_c')    # 5 * 3 节点
d = tf.add(a, b, name = 'add_d')    # 5 + 3 节点
e = tf.add(c, d, name = 'add_e')    # 5 * 3 + (5 + 3) 节点

s = tf.square(a)

In [6]:
# 数据流图的定义
sess = tf.Session()
print(sess.run(c))    # 运行c节点，逐渐计算依赖关系,可以更改运行成别的节点，得到不一样的输出
print(sess.run(s))    # 5 * 5
sess.close()

15
25


In [5]:
# 使用tensorboard
writer = tf.summary.FileWriter('../../test/book/my_graph', sess.graph)    # 将数据流图的描述放置在目录下
writer.close()

### Tensorflow 张量
---
1. 张量 : n维矩阵的抽象
    * 1D张量 : 向量
    * 2D张量 : 2维矩阵
    * 张量可以使用 list / tuple 定义形状
        ```
        shape0 = []        # 定义了0阶张量，标量
        shape1 = [3]       # 定义了1阶张量，向量，长度是3
        shape2 = [3, 2]    # 定义了2D张量，形状是[3, 2]
        shape3 = [None]    # 任意长度的向量
        shape4 = None      # 任意规模的张量
        ```
2. 接收张量的数据流图
    * 简化`input`节点
    * 直接依赖节点变少
    * 灵活性增强，计算速度提升(矩阵运算)
3. 实例

  定义计算公式的数据流图

  $$ y = 5 * 3 + (5 + 3)$$
  
  * 节点
  
      * `tf.reduce_prod` : 累乘张量
      * `tf.reduce_sum` : 累加张量
      * `tf.shape` : 获得张量的形状的Op
  * Tensor对象 = 张量

4. 数据类型
    1. Tensorflow可以接收 **Python数值，bool,str,list**
        * 数值 : 0阶张量（标量）
        * list(1) : 1阶张量
        * list(n) : n阶张量
    2. 常用数据类型,**常用数据类型添加在每一个节点函数的最后声明节点的数据类型,`dtype`**
        1. tf.string
        2. tf.bool
        3. tf.float32 / 64
        4. tf.int64 / 32 / 16
        5. tf.unint8
    3. 使用Numpy手动定义Tensor对象
        
        * 需要注意，tf中的数据类型和numpy中的数据类型中(数值类型和bool类型都是完全一致的，但是string不一致)，我们可以使用numpy定义string数组，但是只要不定义dtype即可传给tensorflow

In [11]:
a = tf.constant([5, 3], name = 'input_a')    # input节点只有一个张量节点
b = tf.reduce_prod(a, name = 'input_b')      # 5 * 3
c = tf.reduce_sum(a, name = 'sum_c')         # 5 + 3
d = tf.add(c, b, name = 'add_d')

In [13]:
sess = tf.Session()
sess.run(d)

23

In [14]:
# numpy and tensor
print(np.int32 == tf.int32, np.int64 == tf.int64, np.float32 == tf.float32, np.float64 == tf.float64,\
     np.bool == tf.bool)
string = np.array(['apple', 'banana', 'pear'])
test_op = tf.constant(string, name = 'fruit')
print(test_op)    # 返回tensor对象

True True True True True
Tensor("fruit:0", shape=(3,), dtype=string)


In [20]:
# 张量的规模
a = tf.constant(np.array([1, 2, 3, 4]), name = 'input_a')
b = tf.shape(a)
sess.run([a, b])
# print(a, b)    # a是一个4长度的向量，b是一个向量代表形状,请使用 sess.run 执行 b 可以查看到结果

[array([1, 2, 3, 4]), array([4], dtype=int32)]

### Tensorflow Operation
---
1. 节点(Op) : 是对Tensor对象执行运算的节点，计算完毕之后会返回0或者多个张量(**返回值是张量的操作句柄**)
2. 节点的分类

    * 计算型节点
    * 特殊节点 : 无输入无输出，执行特定的功能(初始化等等)
3. 属性
    
    * `name` : 标识节点，在tensorboard中可以查询和方便引用
4. 运算符重载 : 无法指定name,需要使用Op制定name
    
    * `-x`, `tf.negative` : 相反数
    * `~x`, `tf.logic_not` : 逻辑非,只适用于bool类型
    * `abs(x)`, `tf.abs` : 绝对值
    * `x | y` : 逻辑或
    * `x ^ y` : 逻辑异或
    * `x & y` : 逻辑与
    * `tf.pow`
    * `< / <= / > / >=`
    * `+ | - | * | / | //` : 浮点数除法，向下取整除法
    * `tf.equal / tf.not_equal` : 判断张量是否相同,返回numpy的bool类型掩码数组,逐元素判断是否相同
    * `==` : 判断两个Tensor对象是不是一个对象，不是判断相同

In [28]:
# ~x
a = tf.constant(np.array(6, dtype = np.bool), name = 'bool')
b = ~a
print(b)

# -x
a = tf.constant(np.array(6, dtype = np.int), name = 'int')
b = - a
print(b)

# abs(x)
a = tf.constant(np.array(-6, dtype = np.int), name = 'abs')
b = abs(a)
print(b)

# tf.equal
a = tf.constant(np.array([1, 2, 3], dtype = np.int), name = 'bool1')
b = tf.constant(np.array([1, 3, 3], dtype = np.int), name = 'bool2')
c = tf.equal(a, b)

sess = tf.Session()
ans = sess.run(c)
print(ans)

# == 
print(a == a)
print(a == b)

Tensor("LogicalNot_6:0", shape=(), dtype=bool)
Tensor("Neg_6:0", shape=(), dtype=int64)
Tensor("Abs_6:0", shape=(), dtype=int64)
[ True False  True]
True
False


### Tensorflow Graph
---
1. Graph作用
    * 划清图之间的界限
    * 让多个图协同工作
2. 注意点
    * 默认存在一张数据流图Graph对象，在其他图的上下文管理之外的节点会被自动的加入到该图中
    * 大多数的程序中，只需要使用默认数据流图即可，除非我们想要进行图之间的协作
3. 方法
    * `tf.Graph` : 创建新的数据流图,获得句柄
    * `tf.Graph.as_default` : 上下文管理器,管理器中加入的节点会被自动的划入图中
    * `tf.get_default_graph` : 获得默认数据流图的句柄

In [43]:
g = tf.Graph()

with g.as_default():
    # 加入g数据流图中
    a = tf.mul(2, 3)

with tf.get_default_graph():
    # 加入默认数据流图中
    pass

### Tensorflow Session
---
1. 会话器Session
    
    * `tf.Session(target, graph, config)`
        1. `target`：　和分布式计算有关，这里暂时不提及
        2. `graph`：　默认是None(默认数据流图)，如果不是在一个with语句块打开的图上下文管理器中打开Session还是建议传入Graph对象
        3. `config`：　设置数据流图的优化参数和日志选项等等
    * `tf.Session.run（fetches, feed_dict）`
        
       使用run方法来计算期望的Tensor对象的输出
       * fetches : 接收任意的数据流图的元素(Op, tensor)
          
          1. fetches 是单个张量
          2. fetches 是张量的列表
       * feed_dict : 用于覆盖tensor对象，使用字典类型指定覆盖方式，覆盖必须要保证覆盖之后的值可以转变或者就是覆盖前的值的类型
           
          1. 目的 : 在测试的时候，可以指定输入的测试值，这样可以保证我们的测试计算的速度，可以不用计算替换之前的一些步骤，直接使用替换的值，加快测试的时间
    * `tf.Session.close` : 释放资源
    * 上下文管理器的会话方式推荐 : session的关闭和开启是自动的
     
2. 新节点对象Op
    
    * `tf.global_variables_initializer` : 
    
        准备要使用的所有Variables对象,但是不输出任何值，换言之输出的结果是None,他的目的实质性某一种特殊的任务，是一个特殊的功能性节点

In [29]:
# 等价构造
sess = tf.Session()
sess = tf.Session(graph = tf.get_default_graph())

In [30]:
# sess.run 计算张量，张量是tf.multiply的输出(输出是张量的句柄),tf便会依次计算b值所有的以来输出节点并顺序执行
a = tf.multiply(2, 3)
b = tf.add(2, 3)
sess = tf.Session()
print(sess.run(a))

print(sess.run([a, b]))

6
[6, 5]


In [31]:
# tf.global_variables_initializer
print(type(sess.run(tf.global_variables_initializer())))

<class 'NoneType'>


In [32]:
# feed_dict
a = tf.add(2, 5)
b = tf.multiply(a, 3)
sess = tf.Session()
print(sess.run(b))
replace_dict = {a: 15}
sess.run(b, feed_dict = replace_dict)    # 使用15替换a中原来的7

21


45

In [23]:
# session 上下文管理器
with tf.Session() as sess:
    # 运行数据流图
    pass
# sess自动关闭

### Tensorflow 占位节点(Op)
---
1. 我们之前的例子中，所有的输入都是我们指定的constant常量，但是我们更多的时候还是希望从用户那里直接获得输入的值，这时候，需要使用占位节点
2. 我们可以利用占位节点创建输入节点
3. 使用方式
    
    * `tf.placeholder(type, shape, name)`
        
        1. `dtype` : 限制和约定了传入数据的数据类型，必须指定
        2. `shape` : 约定了传入 `tensor` 对象的形状，默认是 `None`，可以接收任意形状的张量 `tensor`
        3. `name` : 标识符
    * `feed_dict + tf.placeholder`
    
        1. 必须在 `feed_dict` 中为每一个等待计算的占位节点分配一个键值对，只需要我们加入依赖的占位节点就可以，不依赖的可以不用管它(因为懒惰机制不会计算到)
        2. 不要试图将 `placeholder` 节点加入 `run` 计算，会引发异常,除非你加入了 `feed_dict`

In [35]:
# 创建一个长度是2的向量，数据类型是int32
a = tf.placeholder(tf.int32, shape = [2], name = 'my_input')

b = tf.reduce_prod(a, name = 'prod_b')
c = tf.reduce_sum(a, name = 'sum_c')

d = tf.add(b, c, name = 'add_d')

with tf.Session() as sess:
    # 向量必须长度是2, 类型必须是int32, 否则会引发异常
    input_dict = {a: np.array([5, 3], dtype = np.int32)}
    print(sess.run(d, feed_dict = input_dict))
    try:
        sess.run(a, feed_dict = {a: np.array([5, 3], dtype = np.int32)})
        print("Safe !")
    except Exception as e:
        print("Some error")
        print(e)

23
Safe !


### Tensorflow Variable对象
---
1. 在机器学习中，Tensor对象和Op对象都是不可变动的，但是我们需要一部分参数是变动的保证算法的可执行，Variable中可以保存在动词调用中**可变张量值**
    
    `tf.Variable(data, dtype, name)`
    
2. 作用机制 : 

    1. Variable对象可以用在任何会使用Tensor对象的Tensorflow函数或者Op中，将当前值传给使用它的Op
    
    2. 状态交给tf.Session管理，在使用Variable之前，需要使用Session对Variable对其初始化，保证session开始追踪这些Variable对象的值
    
    3. 不同的session可以使用相同的Variable,但是保留自己会话的备份，Variable值之间不互相影响
    
    4. `tf.Variable(trainable)`
        
        `trainable` 参数可以用来指定我们的optimizer类是否可以修改这些变量来迭代算法，如果只允许手动改动，设定成False
           
3. Variable对象是张量（一般阶数较高） : 
    
    * 全0
    * 全1
    * 随机数填充
    
4. 初始化Variable对象的Op

    每一个Op接收一个shape参数创建指定形状的张量,返回Tensor对象，**将其送入Variable当做参数构造**
    
    * `tf.zeros` : 创建全0的Variable
    * `tf.ones` : 创建全1的Vriable
    * `tf.random_normal(shape, mean, stddev)` : 均值是mean, 标准差是stddev正态分布的随机张量
    * `tf.random_uniform(shape, minval, maxval)` ：　[minval, maxval] 之间均匀分布的随机张量
    * `tf.truncated_normal(shape, mean, stddev)` : 类似normal但是所有的值都不会偏离均值超过 2 * stddev 
    
5. Variable对象的修改

    * `tf.Variable.assign` : 为Variable赋予新的值，**这是一个Op，需要在run中运行才会生效**
    * `tf.Variable.assign_add(number)` : + number
    * `tf.Variable.assign_sub(number)` : - number
    
    * `tf.assign(var_op, new_state)` : 一样的效果
       
6. 初始化Variable对象 [Op]
    
    * `tf.global_variables_initializer` : 初始化全部的Variable对象
    * `tf.variables_initializer` : 初始化部分的Variable对象(列表)

In [40]:
# variable使用方式
sess = tf.Session()
my_var = tf.Variable(3, name = 'my_var')

a = tf.add(5, my_var)
mul = tf.multiply(8, my_var)
sess.run(tf.global_variables_initializer())
sess.run([mul, a])

[24, 8]

In [42]:
# 初始化Op
z = tf.zeros([2, 2])
o = tf.ones([6])
uniform = tf.random_uniform([3, 3, 3], minval = 0, maxval = 10)    # 形状是 3, 3, 3 在 [0, 10]之间均匀分布的随机数
normal = tf.random_normal(shape = [3, 3, 3], mean = 0.0, stddev = 2.0)
trunc = tf.truncated_normal([2, 2], mean = 5.0, stddev = 1.0)

# 包装成Vriable
random = tf.Variable(trunc)
sess.run(tf.global_variables_initializer())
sess.run(random)

array([[ 4.22568321,  3.75998259],
       [ 4.82036686,  6.12445593]], dtype=float32)

In [52]:
# Variable 初始化
# global_variables_initializer : 全部变量初始化
# variables_initializer : 部分变量初始化,使用列表指定初始化的Variable的Op句柄

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

init = tf.variables_initializer([random])

In [44]:
# Variable 的重新赋值

my_var = tf.Variable(1)
# my_var = my_var * 2
my_var_mul_2 = my_var.assign(my_var * 2)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)    # 初始化Variable节点,开始跟踪变量Op对象
    print(sess.run(my_var_mul_2))    # 1 * 2
    print(sess.run(my_var_mul_2))    # 2 * 2
    print(sess.run(my_var_mul_2))    # 4 * 2
    
    print(sess.run(my_var.assign_add(1)))    # 8 + 1
    print(sess.run(my_var.assign_sub(1)))    # 9 - 1
    print(sess.run(tf.assign(my_var, my_var * 3)))    # my_var = my_var * 3 -> 8 * 3

Tensor("Assign_2:0", shape=(), dtype=int32_ref)
2
4
8
9
8
24


In [45]:
# session不共享Variable

my_var = tf.Variable(0)
init = tf.global_variables_initializer()

change_2 = my_var.assign_add(10)

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(change_2))

with tf.Session() as sess:
    # 值不共享
    sess.run(init)
    print(sess.run(change_2))
    sess.run(init)
    print(sess.run(my_var))
    
# trainable
my_var = tf.Variable(0, trainable=False)

10
10
0


### Tensorflow 名称作用域
---
1. Tensorflow中为我们提供了一种帮助用户组织数据流图的机制 : 名称作用域，在可视化的时候也非常的有用
2. 作用
    
    * 名称作用域可以将Op划分到更大的，有名称的语句块中(封装)，使得可视化的效果更好

In [1]:
# 名称作用域
with tf.name_scope("scope_a"):
    a = tf.add(1, 2, name='a_add')
    b = tf.multiply(a, 3, name='a_mul')
    
with tf.name_scope("scope_b"):
    c = tf.add(4, 5, name='b_add')
    d = tf.multiply(c, 6, name='b_mul')

e = tf.add(b, d, name='output')

# 可视化
writer = tf.summary.FileWriter('../../test/book/my_graph/', graph = tf.get_default_graph())
writer.close()

### Tensorflow 基础实践
---
tensorboard 实践函数
  
* `tf.summary.scalar(name, tensor)` : 创建汇总数据
    
    1. name : 在tensorboard中的纵坐标的名称
    
    2. tensor : 是纵坐标的值
    
    P.S. : **一种汇总数据对应于一个二维的坐标图，多个对应多个二维坐标图**
    
    
* `tf.summary.merge_all()` : 合并所有的汇总数据到一个Op中，调用一次可以执行所有的汇总，**所有的summary都需要被run才可以放入磁盘**

* `write.add_summary(summary, global_step)` : **global_step是迭代的横坐标(x)**,summary是写入的summary数据(y,**汇总数据的运行结果**)，**summary数据写入磁盘**

* `writer.flush()` : 刷新写入数据

In [1]:
graph = tf.Graph()

with graph.as_default():
    # 变量空间，这里的变量是全局的，所以我们分配一个单独的变量空间
    with tf.name_scope('variables'):
        # 记录运行次数
        global_step = tf.Variable(0, dtype = tf.int32, trainable = False, name = 'global_step')
        # 记录累和的变化
        total_output = tf.Variable(0.0, dtype = tf.float32, trainable = False, name = 'total_output')
    
    # 变换
    with tf.name_scope('transformation'):
        # 占位Op
        a = tf.placeholder(tf.float32, shape = [None], name = 'input_placeholder_a')
        
        with tf.name_scope("hidden_layer"):
            b = tf.reduce_prod(a, name='product_b')
            c = tf.reduce_sum(a, name='sum_c')
        
        # 输出空间
        with tf.name_scope("output"):
            output = tf.add(b, c, name='output')
        
    # 更新变量空间的值
    with tf.name_scope('update'):
        # 变更的计算
        update_total = total_output.assign_add(output)
        increment_step = global_step.assign_add(1)
        
    # tensorboard 汇总数据
    with tf.name_scope("summaried"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name = 'average')
        # 创建tensorboard汇总数据
        tf.summary.scalar('Output', output)
        tf.summary.scalar('sum_of_output', update_total)
        tf.summary.scalar('avg', avg)
    
    # 初始化
    with tf.name_scope('init'):
        init = tf.global_variables_initializer()
        # 合并tensorboard的汇总数据到一个Op中
        merge_summ = tf.summary.merge_all()

In [2]:
sess = tf.Session(graph = graph)
writer = tf.summary.FileWriter('../../test/book/my_graph', graph)
sess.run(init)

def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merge_summ], feed_dict = feed_dict)
    writer.add_summary(summary, global_step=step)

run_graph([2, 8])
run_graph([3, 1, 3, 3])
run_graph([8])
run_graph([7, 3, 1])
run_graph([6, 3])
run_graph([0, 2])
run_graph([4, 5, 6])

writer.flush()    # 汇总全部写入磁盘
writer.close()
sess.close()